# Advection diffusion cas test pour un champ sur l'interface FT


## Introduction
 
 Validation made by: Antoine du Cluzeau

 Report generated 31/01/2024

In [ ]:
from trustutils import run 
from pathlib import Path
 
run.TRUST_parameters("1.9.4_beta")

In [ ]:
run.reset()
run.initBuildDirectory()  # copy src/ in build/
run.addCase(".", "diffusion.data", nbProcs=8) 
run.addCase(".", "advection_croissance.data", nbProcs=8) 
run.addCase(".", "advection_decroissance.data", nbProcs=8) 
run.printCases()
run.runCases()

## Computer Performance

In [ ]:
run.tablePerf()

## Resultats


In [ ]:
from trustutils import visit
from math import *
import re
import os
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import sys
from collections import Counter
import matplotlib.patches as mpatches
print(os.getcwd())
sys.path.append(os.getcwd())
from Fields import *
import glob
import turtle

In [ ]:
def param_image(name, grid=True, legend=False, legend_out=True, n_dpi=100, fontsizelegend=10):
    scalepng=3.5       
    inSizeLegend=int(scalepng*3.5)
    if (grid):
        plt.grid()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.tick_params(axis='x', labelsize=12)
    plt.tick_params(axis='y', labelsize=12)
    if (legend_out):
    	plt.legend(fontsize=fontsizelegend, loc='center left', bbox_to_anchor=(1.05, 0.5))
    else:
    	plt.legend(fontsize=fontsizelegend, loc=0,prop={'size':inSizeLegend}, frameon=True, ncol=1)
    	plt.legend(fontsize=fontsizelegend).set_visible(legend) ### deactive la legende

On réalise un cas test de diffusion pure d'un champ FT :
\begin{equation}
\frac{\partial\Gamma}{\partial t} = D_s\nabla_s^2\Gamma
\end{equation}
On simule pour cela une bulle au repos dans un écoulement au repos.
On impose une quantité initiale de surfactant sous la forme : 
\begin{equation}
\Gamma(\theta)=0.5(1-cos\theta)
\end{equation}
où $\theta$ est l'angle entre la verticale et le segment allant du centre de masse de la bulle jusqu'à un point de l'interface. La solution analytique s'écrit alors :
\begin{equation}
\Gamma(\theta, t)=0.5(1-e^{-2t*/Pe_s}cos\theta)
\end{equation}
où $Pe_s = Ud_b/D_s$ et $t^*=tU/d_b$.
On postraite ici le maximum de concentration pour le comparer la valeur théorique :
\begin{equation}
\Gamma_{max}(t)=0.5(1-e^{-2t*/Pe_s})
\end{equation}

In [ ]:
Surfactant_max=Field.LoadFromFile("build/diffusion_bulles_surfactant_max.out", 0, 1)
Surfactant_max_theorique=Field.LoadFromFile("build/diffusion_bulles_surfactant_max.out", 0, 1)
for i in range(len(Surfactant_max_theorique)):
    Surfactant_max_theorique.npa[i] = 0.5 * (1.+exp(-2*Surfactant_max_theorique.ax[i]*1.e-7/(0.0001*0.0001)))

plt.figure()
plt.ylabel("quantite max", fontsize=15)
plt.xlabel(r"$t$", fontsize=15)
plt.scatter(Surfactant_max.ax, Surfactant_max.npa, color='red', label="DNS", marker='x')
plt.plot(Surfactant_max_theorique.ax, Surfactant_max_theorique.npa, color='black', label="ref", linestyle='-')
param_image("Diffusion.png", legend=True, legend_out=False)
plt.show()


On réalise un cas test d'advection pure d'un champ FT :
\begin{equation}
\frac{\partial\Gamma}{\partial t} + \nabla_s.(\Gamma u_s)= 0
\end{equation}
On simule pour cela une bulle au repos dans un écoulement au repos.
La bulle va alors être soumise à une variation de son volume (croissant et décroissant).
L'on doit s'assurer alors de la conservation de la quantite de surfactant sur l'interface telle que :
\begin{equation}
\Gamma(t)=\frac{A_0}{A(t)}\Gamma_0
\end{equation}
où $A$ est la surface de la bulle à un instant donné, et $\Gamma_0$ la concentration initiale de surfactant.

In [ ]:
Surfactant_c=Field.LoadFromFile("build/advection_croissance_bulles_surfactant.out", 0, 1)
Surfactant_ref_c=Field.LoadFromFile("build/advection_croissance_bulles_surfactant.out", 0, 1)
Surfactant_ref_c[:]=Surfactant_ref_c.npa[0]
Surface_c=Field.LoadFromFile("build/advection_croissance_bulles_surface.out", 0, 1)
concentration_ref_c =Surfactant_ref_c/Surface_c
concentration_c=Surfactant_c/Surface_c
Surfactant=Field.LoadFromFile("build/advection_decroissance_bulles_surfactant.out", 0, 1)
Surfactant_ref=Field.LoadFromFile("build/advection_decroissance_bulles_surfactant.out", 0, 1)
Surfactant_ref[:]=Surfactant_ref.npa[0]
Surface=Field.LoadFromFile("build/advection_decroissance_bulles_surface.out", 0, 1)
concentration_ref =Surfactant_ref/Surface
concentration=Surfactant/Surface

plt.figure()
plt.ylabel("concentration", fontsize=15)
plt.xlabel(r"$t$", fontsize=15)
plt.xlim(0.,0.01)
plt.ylim(0.,1.5)
plt.scatter(concentration_c.ax, concentration_c.npa, color='red', label="DNS volume croissant", marker='x')
plt.plot(concentration_ref_c.ax, concentration_ref_c.npa, color='red', label="ref", linestyle='-')
plt.scatter(concentration.ax, concentration.npa, color='blue', label="DNS volume decroissant", marker='x')
plt.plot(concentration_ref.ax, concentration_ref.npa, color='blue', label="ref", linestyle='-')
param_image("Advection.png", legend=True, legend_out=False)
plt.show()

On s'assure par vérification visuel de la régularité du champ en fin de calcul que tout s'est bien déroulé.
On observe en particulier qu'il n'y a pas de diffusion numérique associée au processus de remaillage lorsque l'interface gonfle ou dégonfle. Cela permet de valider l'algorithme pour le calcul des intersections des triangles et la redistribution conservative, et non diffusive, dans les phases de correction de volume, de barycentrage, d'ajout et de suppression de sommets. 

In [ ]:
visit.showField("diffusion.lata", "Pseudocolor", "CONCENTRATION_INTERFACE_ELEM_INTERFACES", plotmesh=False)

In [ ]:
visit.showField("advection_croissance.lata", "Pseudocolor", "CONCENTRATION_INTERFACE_ELEM_INTERFACES", plotmesh=False)

In [ ]:
visit.showField("advection_decroissance.lata", "Pseudocolor", "CONCENTRATION_INTERFACE_ELEM_INTERFACES", plotmesh=False)